# How to run

You can run each cell in order with shift+enter, or run the whole thing at once by using the Kernel menu up above and hitting 'Restart & Run All'

This notebook expects a folder structure as follows
```
---
 | traps.ipynb
 |-data\Orcus - Traps.xlsx
 |-output
```

and will take data\Orcus - Traps.xlsx and create a markdown file output\traps.md

We use Pandas to read in the excel spreadsheet, and define a function which takes a row in the Pandas DataFrame and puts all the values in the right place (with a little logic to handle fields which aren't always there) to create a markdown formatted string.

It's pretty similar to what the existing Mail Merge does, but we have complete control over how we build the string and map values. The key Pythonic bit is the curly braces {} and the `**dict`. Curly braces indicate a place that we want to input a value in a string with a **key**, and ** takes a dictionary and puts the values with the proper keys in the proper place.  

```
'{hello}'.format(**{'hello':'world', 'not_needed':'mars'})
evaluates to
'world'
```


Then we join that list of strings and save a file.  For more on Python, I recommend [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/)

In [1]:
# cell 0, standard imports

import pandas as pd
import os

dot  = '●'

In [2]:
#cell 1, read in data from filename

filename = 'Orcus - Traps.xlsx'

#by default Pandas loads missing values with ugly NaNs, so we replace those with the empty string ''
data = pd.read_excel(os.path.join('data', filename)).fillna('')
data.head()

,Name,Description,Level,Role,Type,XP,Skill,Notice,Trigger,Countermeasures,Initiative,Action,Range 1,Target,Attack,Hit,Miss,Effect,Special
0,Quicksand,A patch of quicksand across one or several squ...,1,Hazard,Blocker,50,Nature,DC 18: The character notices that the ground a...,The trap attacks when a character enters one o...,A character in the pit can rise 1 square as a ...,,,Melee,The creature that entered the trigger square.,+4 vs Reflex,The target falls into the pit and sinks 1 squa...,Target returns to last square it occupied and ...,,
1,Stinking Trench,"A thin (one-square wide, one-square deep) tren...",1,Hazard,Blocker,50,Perception,DC 5: The character notices the horrific smell...,The trap is triggered when a creature enters o...,A character who makes a successful Athletics c...,,,Melee,The creature.,+4 vs Fortitude,The target is dazed (save ends).,,,
2,Simple Spear Trap,A spear shoots out from a hole near to the tri...,1,One-Off,Alarm,25,Perception,DC 12: The character notices the mechanism tha...,The trap attacks when a character comes into c...,An adjacent character can disable the spear tr...,,,Melee,The creature that triggers the trap.,+4 vs AC,1d8+3 damage.,,,
3,Cinder Trap,One sconce is set up to launch its torch at a ...,1,One-Off,Alarm,25,Perception,DC 12: The character notices the slow-burning ...,The trap attacks when a character comes into c...,An adjacent character can disable the trap wit...,,,Ranged 5,The creature that triggers the trap.,+4 vs Reflex,1d6+1 fire damage.,,,
4,Snare Trap,A noose lays on the ground. When it is disturb...,1,One-Off,Elite Alarm,50,Perception,DC 12: The character notices coiled rope.\nDC ...,The trap attacks when a character enters the s...,A character who makes a successful Athletics c...,,,Melee,The creature that triggers the trap.,+4 vs Reflex,The target is restrained and suspended upside-...,,,


In [3]:
#cell 2, examine 5 random traps

data.sample(5)

,Name,Description,Level,Role,Type,XP,Skill,Notice,Trigger,Countermeasures,Initiative,Action,Range 1,Target,Attack,Hit,Miss,Effect,Special
5,Whistling Staircase,A staircase where the stairs are attached by a...,1,One-Off,Alarm,25,Perception,DC 12: The character notices the cords running...,The trap is triggered when a Medium or larger ...,An adjacent character can disable the whistlin...,,,,,,,,"The trapped stairs make a terrible racket, ale...",
6,Concealled Pit Trap,A pit trap of one or more squares in area is h...,2,Hazard,Alarm,63,Perception,DC 13: The character notices that the squares ...,The trap attacks when a character enters one o...,An adjacent character can trigger the trap wit...,,,Melee,The creature that entered one of the trap’s sq...,+6 vs Reflex,"The target falls into the pit, takes 1d10+3 da...",The target returns to the last square it occup...,The pit trap opens and the trap is no longer h...,
10,Gate Spikes,"When disturbed, spikes shoot out of the top of...",2,Hazard,Assassin,63,Perception,DC 13: The character notices the hidden mechan...,The trap is triggered when a character attempt...,An adjacent character can disable the spears w...,,,Melee,The creature climbing over the gate.,+7 vs AC,1d8+3 damage.,,,
20,Mosaic Trap,The characters must press certain tiles in thi...,8,One-Off,Shooter,100,Perception,DC 17: The character notices that the mosaic t...,The trap attacks when a character comes into c...,An adjacent character can disable the trap wit...,,,Near burst 5,All creatures.,+11 vs Reflex,"3d8+4 damage, and the target takes persistent ...",,,
0,Quicksand,A patch of quicksand across one or several squ...,1,Hazard,Blocker,50,Nature,DC 18: The character notices that the ground a...,The trap attacks when a character enters one o...,A character in the pit can rise 1 square as a ...,,,Melee,The creature that entered the trigger square.,+4 vs Reflex,The target falls into the pit and sinks 1 squa...,Target returns to last square it occupied and ...,,


In [4]:
#cell 3
#we define a function which takes in a row of this trap dataframe and returns an md string

def trap_to_md(row):
    
    #converting the row to a dictionary let's use do some cool stuff with placing values in strings using .format(**row_dict)
    row_dict = dict(row)
    
    #remove leading and trailing whitespace
    for k, v in row_dict.items():
        row_dict[k] = str(v).strip()
    
    # we use completion to check a few edge cases
    completion = dict()
    for key,value in row.items():
        if value:
            completion[key] = value
    
    if row_dict.get('Name') == 'Chapter':
        md = '# {List}  '.format(**row_dict)
#        if row['Details']:
#            md += "\n\n{Details}  ".format(**row_dict)
        return md
    
   
    # we check if nothing is filling in and return an empty row
    if list(completion.keys()) == []:
        md = ''
        return md
    
    #start with name. md is the string we'll be outputing
    md = "### {Name}  ".format(**row_dict)
    
    #add flavor if it exists
    
    if row['Description']:
        md += '\n*{Description}*  '.format(**row_dict)
    
    #'\n' is Python for new line
    if row['Level']:
        md += "\n\n**Level {Level} {Role} {Type}** ({XP} XP Trap)  ".format(**row_dict)
        
    #modify tags a little, since they aren't always present    
    if row_dict.get('Tags'):
        row_dict['Tags'] = '● **{Tags}**'.format(**row_dict)
    
    if row['Skill']:
        md += '\n\n{Skill} {Notice}  '.format(**row_dict)
    
    #if a field exists, we add it
    if row['Trigger']:
        md += '\n\n**Trigger** {Trigger}  '.format(**row_dict)
                
    if row['Action']:
        md += '\n> {Action} Action  '.format(**row_dict)

    if row['Initiative']:
        md += '\n> **Initiative** +{Initiative}  '.format(**row_dict)

    if row['Target']:
        md += '\n> **Target** {Target}  '.format(**row_dict)
    
    if row['Attack']:
        md += '\n> **Attack** {Range 1}; {Attack}  '.format(**row_dict)

    if row['Hit']:
        md += '\n> **Hit** {Hit}  '.format(**row_dict)

    if row['Miss']:
        md += '\n> **Miss** {Miss}  '.format(**row_dict)

    md += '  '.format(**row_dict)
        
    if row['Effect']:
        md += '\n> **Effect** {Effect}  '.format(**row_dict)
        
    if row['Special']:
        md += '\n> **Special** {Special}  '.format(**row_dict)
        
    if row['Countermeasures']:
        md += '\n\n**Countermeasures** \n{Countermeasures}  '.format(**row_dict)
        
    #finally we remove the empty tags
    return md.replace('****','')


In [5]:
#cell 4
#this cell converts the dataframe into a list of markdown formatted strings using the function defined above

output = []

for idx, row in data.iterrows():
    md = trap_to_md(row)
    if md: #this removes empty lines, which you may not want
        output.append(md)

In [6]:
#cell 5
#we can look at elements here using Python list slicing

for elem in output[100:110]:
    print(elem)
    print('')

In [7]:
#cell 6
# now we have to save to disk

fname =  'Orcus Traps.md'

with open(os.path.join('output', fname), 'w', encoding='utf-8') as f:
    f.write('\n\n'.join(output))